# CENG3526 IR MIDTERM HW

To use this code, upload the appropirate *.zip file to colab.

Example code expects the "30Columnists.zip" file be uploded and ready in the GCE backend. 

To do that, you can also mount your **google drive** and use the datasets from there.

Also you can use the **git repo** to download the zip files: 

https://github.com/dtaner/CENG3526/blob/master/30Columnists.zip

or check out (clone) the repo to use datasets.

```
# Clone the entire repo.
!git clone -l -s https://github.com/dtaner/CENG3526.git cloned-repo
%cd cloned-repo
!ls
```

# OGUZHAN SAHIN 160709027
 




IR System was created. TF-IDF is used for scoring documents.

In [1]:
import io, os
import re as re
import zipfile as zipfile
import numpy as np
import nltk
import time
def read_file(zipname):   
    data = [] 
    with zipfile.ZipFile(zipname) as z:
        for zipinfo in z.infolist():
            mytextzip = ''
            if zipinfo.filename.endswith('.txt') and re.search('raw_texts', zipinfo.filename):
                with z.open(zipinfo) as f:
                    textfile = io.TextIOWrapper(f, encoding='cp1254', newline='')
                    for line in textfile:
                        if len(line.strip()): mytextzip += ' ' + line.strip()   
                    data.append(mytextzip)
    return data
kose_yazilari = read_file('30Columnists.zip')

In [2]:
_STOP_WORDS = frozenset([
'a', 'about', 'above', 'above', 'across', 'after', 'afterwards', 'again', 
'against', 'all', 'almost', 'alone', 'along', 'already', 'also','although',
'always','am','among', 'amongst', 'amoungst', 'amount',  'an', 'and', 'another',
'any','anyhow','anyone','anything','anyway', 'anywhere', 'are', 'around', 'as',
'at', 'back','be','became', 'because','become','becomes', 'becoming', 'been', 
'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 
'between', 'beyond', 'bill', 'both', 'bottom','but', 'by', 'call', 'can', 
'cannot', 'cant', 'co', 'con', 'could', 'couldnt', 'cry', 'de', 'describe', 
'detail', 'do', 'done', 'down', 'due', 'during', 'each', 'eg', 'eight', 
'either', 'eleven','else', 'elsewhere', 'empty', 'enough', 'etc', 'even', 
'ever', 'every', 'everyone', 'everything', 'everywhere', 'except', 'few', 
'fifteen', 'fify', 'fill', 'find', 'fire', 'first', 'five', 'for', 'former', 
'formerly', 'forty', 'found', 'four', 'from', 'front', 'full', 'further', 'get',
'give', 'go', 'had', 'has', 'hasnt', 'have', 'he', 'hence', 'her', 'here', 
'hereafter', 'hereby', 'herein', 'hereupon', 'hers', 'herself', 'him', 
'himself', 'his', 'how', 'however', 'hundred', 'ie', 'if', 'in', 'inc', 
'indeed', 'interest', 'into', 'is', 'it', 'its', 'itself', 'keep', 'last', 
'latter', 'latterly', 'least', 'less', 'ltd', 'made', 'many', 'may', 'me', 
'meanwhile', 'might', 'mill', 'mine', 'more', 'moreover', 'most', 'mostly', 
'move', 'much', 'must', 'my', 'myself', 'name', 'namely', 'neither', 'never', 
'nevertheless', 'next', 'nine', 'no', 'nobody', 'none', 'noone', 'nor', 'not', 
'nothing', 'now', 'nowhere', 'of', 'off', 'often', 'on', 'once', 'one', 'only',
'onto', 'or', 'other', 'others', 'otherwise', 'our', 'ours', 'ourselves', 'out',
'over', 'own','part', 'per', 'perhaps', 'please', 'put', 'rather', 're', 'same',
'see', 'seem', 'seemed', 'seeming', 'seems', 'serious', 'several', 'she', 
'should', 'show', 'side', 'since', 'sincere', 'six', 'sixty', 'so', 'some', 
'somehow', 'someone', 'something', 'sometime', 'sometimes', 'somewhere', 
'still', 'such', 'system', 'take', 'ten', 'than', 'that', 'the', 'their', 
'them', 'themselves', 'then', 'thence', 'there', 'thereafter', 'thereby', 
'therefore', 'therein', 'thereupon', 'these', 'they', 'thickv', 'thin', 'third',
'this', 'those', 'though', 'three', 'through', 'throughout', 'thru', 'thus', 
'to', 'together', 'too', 'top', 'toward', 'towards', 'twelve', 'twenty', 'two', 
'un', 'under', 'until', 'up', 'upon', 'us', 'very', 'via', 'was', 'we', 'well', 
'were', 'what', 'whatever', 'when', 'whence', 'whenever', 'where', 'whereafter',
'whereas', 'whereby', 'wherein', 'whereupon', 'wherever', 'whether', 'which', 
'while', 'whither', 'who', 'whoever', 'whole', 'whom', 'whose', 'why', 'will', 
'with', 'within', 'without', 'would', 'yet', 'you', 'your', 'yours', 'yourself',
'yourselves', 'the','.',',','?','!'])

In [3]:
def get_tokenize(text):       
    acronym_each_dot = r"(?:[a-z]\.){2,}"
    acronym_end_dot = r"\b[a-z]{2,3}\."
    suffixes = r"[a-z]{3,}' ?[a-z]{0,3}"
    numbers = r"\d+[.,:\d]+"
    any_word = r"[a-z]+"
    punctuations = r"[a-z]*[.,!?;:]"
    word_regex = "|".join([acronym_each_dot,
                           acronym_end_dot,
                           suffixes,
                           numbers,
                           any_word,
                           punctuations])    
    return re.compile("%s"%word_regex, re.I).findall(text)

In [4]:
def clean_text(corpus):
    stem =nltk.PorterStemmer()
    cleaned_words = []   
    for text in corpus:
        text = text.lower()
        text = get_tokenize(text)
        text = [y for y in text if not y in _STOP_WORDS]    
        text = [stem.stem(word)for word in text]
        text =" ".join(text)
        cleaned_words.append(text)         
    return cleaned_words
cleaned_text = clean_text(kose_yazilari)
del(kose_yazilari) # for memory allocation


In [5]:
def create_word_freq(corpus):
    word_freq=dict()
    for index,document in enumerate(corpus):
        tokens = get_tokenize(document)
        for unique_token in tokens:
            if unique_token in word_freq.keys():
                word_freq[unique_token]+=":"+str(index)
            else:
                word_freq[unique_token]=str(index)
    return word_freq
word_freq = create_word_freq(cleaned_text)

In [6]:
def tfidf_scores(word_freq,query,postingList):
    docId_located_query = word_freq[query].split(":")
    idfScore = len(postingList) / len(docId_located_query) + 1   
    tfIdfScoreTotal = dict()
    for docId in docId_located_query:
        counter = 0
        tokens = get_tokenize(postingList[int(docId)])
        for token in tokens:
            if token == query:
                counter +=1
        tfscore = counter / len(tokens)
        tfIdfScoreTotal[docId] = tfscore * idfScore
    return tfIdfScoreTotal

In [7]:
def search(word_freq,query,postingList):
    query = query.lower()
    search_result = dict()
    for word in get_tokenize(query):
        tfidfscore=tfidf_scores(word_freq,word,postingList)
        for j in tfidfscore.keys():
                if j in search_result.keys():
                    search_result[j] += tfidfscore[j]
                else :
                    search_result[j] = tfidfscore[j]
    search_result = sorted(search_result.items(), key = lambda x:x[1],reverse=True)
    return search_result


In [8]:
query = input("Search word that you looking for : ")
result=search(word_freq,query,cleaned_text)
print(result)  

Search word that you looking for : hey
[('974', 0.7003168854685378), ('973', 0.42746615087040624), ('971', 0.3291489361702128), ('997', 0.32589003581209186), ('955', 0.32269503546099293), ('340', 0.227522306108442), ('989', 0.22391084093211752), ('967', 0.22239792984473838), ('958', 0.21797942792729322), ('962', 0.21099290780141844), ('969', 0.20317835566062517), ('990', 0.20317835566062517), ('999', 0.19476268412438624), ('993', 0.17696179363989936), ('970', 0.16708067825899126), ('978', 0.16708067825899126), ('954', 0.15453001698132054), ('982', 0.1476004198072703), ('1031', 0.14694148936170212), ('984', 0.14126563784129303), ('965', 0.14006337709370756), ('968', 0.13657632206232895), ('995', 0.13272134522992451), ('1181', 0.11310960005849237), ('686', 0.09168494043738518), ('31', 0.07912234042553193), ('404', 0.07762946607788036), ('39', 0.07690395704911512), ('1396', 0.07429998559147015), ('1355', 0.07018100984439504), ('626', 0.07003168854685378), ('346', 0.06944070383337822), ('1